# Modèle autorégressif (AR)
---
## 1. Formulation mathématique

Un modèle AR ou autorégressif est un modèle de régression pour les séries temporelles (variables qui évoluent au cours du temps) **stationnaire uniquement**, i.e. $\mathbb{E}(X_{t}) = cste$. Ce modèle base ses estimations sur la relation linéaire entre la valeur à prédire (future) et les valeurs observées (passé).

Un modèle AR d'ordre $p$, noté $AR(p)$ se modélise sous la forme suivante :

$$
AR(p) = c + \phi_{1}X_{t-1} + ... + \phi_{n}X_{t-p} + \epsilon_{t} = c + \epsilon_{t} + \sum_{i = 1}^{p}\phi_{i}X_{t-i}
$$

Avec :

1. $X_{t-i}$ : valeurs de la série à l'instant $t - i$ supposées connues. Entre autres, $X = (X_{t-p},..,X_{t - 1}) \in \mathbb{R^{p}}$
2. $(\phi{i})_{i \in \mathbb{N}} \in \mathbb{R}$ : coefficient à déterminer
3. $c \in \mathbb{R}$ une constante
4. $\epsilon_{t} \in \mathbb{R}$ un bruit
 
En d'autres termes, on prédit la valeur $X_{t}$ en fonction des valeurs précédentes $(X_{t-i})_{i < p}$ supposées connues. **L'objectif est de déterminer les coefficients $(\hat{\phi_{i}})_{i \in \mathbb{N}}$ de telle sorte à trouver l'estimateur $\hat{X_{t}}$ de $X_{t}$.** Il existe plusieurs méthodes pour y arriver.

---
## Méthodes de résolution
### 2.1 Méthode des moindres carrés ordinaires

On estime les $(\hat{\phi{i}})_{i \in \mathbb{N}}$ qui minimisent leurs erreur quadratique. L'erreur moyenne obtenue sur les échantillons $(\hat{\phi_{i}})_{i \in \mathbb{N}}$ notée $ELMQ$ est définie par :

$$
ELMQ((\hat{\phi{i}})_{i \in \mathbb{N}}) = \sum_{i = 1}^{n}(\phi_{i} - \hat{\phi_{i}})^{2}
$$

On se ramène à un problème **d'optimisation** :


### 2.2 Les équations de Yule-Walker

On peut trouver les $(\hat{\phi_{i}})_{i \in \mathbb{N}}$ grâce aux coefficients d'autocovariance estimés $(\hat{\gamma_{X}}(i))_{i \in \mathbb{N}}$ définis par :

1. Si le signal de longueur $T$ n'est pas centré, i.e. $\mathbb{E}(X) \neq 0$:
$$
\hat{\gamma_{X}}(i) = \frac{1}{T}\sum_{t=i+1}^{T}(X_{t} - \bar{X})(X_{t-i} - \bar{X})
$$

2. Si le signal de longueur $T$ est centré, i.e. $\mathbb{E}(X) = 0$ :

$$
\hat{\gamma_{X}}(i) = \frac{1}{T}\sum_{t=i+1}^{T}X_{t}X_{t-i}
$$

Avec $\bar{X}$ la moyenne empirique du signal $\bar{X} = \frac{1}{T}\sum_{t=1}^{T}X_{t}$

Les **équations de Yule-Walker** sont de la forme :

1. Calcul des $(\hat{\phi_{i}})_{i \in \mathbb{N}}$
$$
    \begin{bmatrix}
        \hat{\gamma_{X}}(1) \\
        \hat{\gamma_{X}}(2) \\
        \vdots \\
        \hat{\gamma_{X}}(p)
    \end{bmatrix} =
    \begin{bmatrix}
    \hat{\gamma_{X}}(0) & \hat{\gamma_{X}}(1) & \dots & \hat{\gamma_{X}}(p-1) \\
    \hat{\gamma_{X}}(1) & \hat{\gamma_{X}}(0) & \dots & \hat{\gamma_{X}}(p-2) \\
    \vdots & \vdots & \vdots & \vdots \\
    \hat{\gamma_{X}}(p-1) & \hat{\gamma_{X}}(p-2)& \dots & \hat{\gamma_{X}}(0) \\
    \end{bmatrix}
    \begin{bmatrix}
    \hat{\phi_{1}} \\
    \hat{\phi_{2}} \\
    \vdots \\
    \hat{\phi_{p}} \\
    \end{bmatrix}
$$

La matrice dans l'équation est appelée matrice d'autocovariance. C'est une matrice de type Toeplitz, ce qui permet d'utiliser l'algorithme de Levinson-Durbin pour la résolution du système une fois les coefficients d'autocovariance estimés.

2. Calcul de $Var(\epsilon_{t})$ :

$$
Var(\epsilon_{t}) = \sigma^{2} = \hat{\gamma_{X}}(0) - \sum_{i=1}^{p}\hat{\phi_{i}}\hat{\gamma_{X}}(i)
$$

---
#### Interlude sur l'algorithme de Levinson-Durbin

Au lieu de de résoudre le système en inversant la matrice d'autocovariance, Levinson-Durbin propose de construire les coefficients AR $(\hat{\phi_{i}})_{i \in \mathbb{N}}$ de manière récursive grâce à des coefficients $\alpha_{j}^{(i)}$ tel que :

$$
\alpha_{i}^{(i)} = \alpha_{i}= \hat{\phi_{i}}
$$

Attention : pour $j \neq i,  \alpha_{j}^{(i)} \neq \hat{\phi_{i}}$. On trouve donc les $(\hat{\phi_{i}})_{i \in \mathbb{N}}$ à l'aide des $(\alpha_{j}^{(i)})_{i,j \in \mathbb{N}}$ grâce à la relation suivante :

$$
\alpha_{i} = \frac{\hat{\gamma_{X}}(i) - \sum_{j=1}^{i-1} \alpha_{j}^{(i-1)}\hat{\gamma_{X}}(i-j)}{E_{i-1}}
$$

Avec les relations de récurrence suivantes :

$$
\alpha_{j}^{(i)} = \alpha_{j}^{(i-1)} - \alpha_{i}\alpha_{i-j}^{(i-1)} \textcolor{white}{.....} (1)
$$
Pour  $i \in ⟦1; p⟧, j \in ⟦1; i-1⟧$ avec $\alpha_{i}^{(i)} = \alpha_{i}$ et $\alpha_{1}^{(1)} = \frac{\hat{\gamma_{X}}(1)}{\hat{\gamma_{X}}(0)}$
$$
E_{i} = E_{i-1}(1-\alpha_{i}^{2}) \textcolor{white}{.....} (2)
$$
Pour $i \in ⟦1; p⟧$ avec $E_{1} = \hat{\gamma_{X}}(0)(1-(\alpha_{1}^{(1)})^{2})$ 

On a finalement que les $(\alpha_{i}^{(p)})_{i \in \mathbb{N}}$ sont les $(\hat{\phi_{i}})_{i \in \mathbb{N}}$ et $E_{p}$ est l'estimation du bruit blanc $\sigma^{2}$

Proposition de l'algorithme de Levinson-Durbin en Python :


---

### 2.3 Calcul de la constante $c$

